# IMPORT LIBRARIES AND LOAD THE WEATHER AND COORDINATES DATA

In [28]:
pip install citipy

In [31]:
pip install pyproj

   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB 2.0 MB/s eta 0:00:03
   - -------------------------------------- 0.2/6.1 MB 1.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.4/6.1 MB 2.9 MB/s eta 0:00:02
   --- ------------------------------------ 0.6/6.1 MB 3.5 MB/s eta 0:00:02
   ------- -------------------------------- 1.1/6.1 MB 5.4 MB/s eta 0:00:01
   --------------- ------------------------ 2.3/6.1 MB 8.6 MB/s eta 0:00:01
   --------------------------- ------------ 4.1/6.1 MB 13.2 MB/s eta 0:00:01
   ---------------------------------------  6.0/6.1 MB 16.7 MB/s eta 0:00:01
   ---------------------------------------  6.1/6.1 MB 16.9 MB/s eta 0:00:01
   ---------------------------------------- 6.1/6.1 MB 14.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [38]:
pip install "cartopy"

     ---------------------------------------- 0.0/56.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/56.0 kB ? eta -:--:--
     -------------------- ----------------- 30.7/56.0 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 56.0/56.0 kB 415.7 kB/s eta 0:00:00
   ---------------------------------------- 0.0/10.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.9 MB 5.9 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.9 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.9 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.9 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.9 MB 9.5 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.9 MB 9.5 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.9 MB 4.4 MB/s eta 0:00:03
   --------- ------------------------------ 2.5/10.9 MB 6.5 MB/s eta 0:00:02
   ------

In [39]:
pip install geoviews

INFO: pip is looking at multiple versions of panel to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/557.0 kB ? eta -:--:--
   -------------- ------------------------- 204.8/557.0 kB 6.3 MB/s eta 0:00:01
   ---------------------------------------  553.0/557.0 kB 7.0 MB/s eta 0:00:01
   ---------------------------------------- 557.0/557.0 kB 5.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/7.0 MB ? eta -:--:--
   ---- ----------------------------------- 0.8/7.0 MB 16.8 MB/s eta 0:00:01
   ----------- ---------------------------- 1.9/7.0 MB 17.7 MB/s eta 0:00:01
   ------------------ --------------------- 3.2/7.0 MB 18.6 MB/s eta 0:00:01
   -------------------------- ------------- 4.6/7.0 MB 19.5 MB/s eta 0:00:01
   ------------------------------- -------- 5.4/7.0 MB 18.3 MB/s eta 0:00:01
   -------------------------------------- - 6.8/7.0 MB 18.9 MB/s eta 0:00:01
   -------------

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import numpy as np
import os
import json
import warnings
warnings.filterwarnings('ignore')
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,taoudenni,22.6783,-3.9836,41.96,6,14,3.41,ML,1720468391
1,1,waitangi,-43.9535,-176.5597,8.23,82,100,1.34,NZ,1720468393
2,2,yellowknife,62.4560,-114.3525,23.56,51,20,3.09,CA,1720468395
3,3,hirara,24.8000,125.2833,26.92,94,4,3.09,JP,1720468398
4,4,port-aux-francais,-49.3500,70.2167,2.29,85,9,14.57,TF,1720468401


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City", 
)

# Display the map
city_humidity_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [11]:
# Narrow down cities that fit criteria and drop any results with null values
temp_condition = ((city_data_df["Max Temp"] > 20) & (city_data_df["Max Temp"] < 27))
wind_condition = (city_data_df["Wind Speed"] < 5)
cloud_condition = (city_data_df["Cloudiness"] < 10)

city_data_ND = city_data_df.loc[(temp_condition & wind_condition & cloud_condition), :]

# Drop any rows with null values
city_final_narrow = city_data_ND.dropna()

# Display sample data
city_final_narrow

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,hirara,24.8000,125.2833,26.92,94,4,3.09,JP,1720468398
58,58,fort st. john,56.2499,-120.8529,25.94,41,0,3.60,CA,1720468526
72,72,constantia,44.1833,28.6500,24.38,51,0,2.73,RO,1720468272
90,90,colonia,50.9333,6.9500,20.63,65,0,1.54,DE,1720468599
111,111,uruzgan,32.8333,66.0000,24.70,13,0,0.76,AF,1720468644
148,148,litochoro,40.1006,22.4978,24.27,46,4,1.03,GR,1720468736
150,150,gamba,-2.6500,10.0000,20.96,87,9,2.17,GA,1720468741
198,198,dingli,35.8614,14.3822,25.45,73,0,0.51,MT,1720468858
203,203,mohammedia,33.7066,-7.3944,22.11,88,0,3.60,MA,1720468873
226,226,isola di capo rizzuto,38.9600,17.0933,24.40,81,0,1.44,IT,1720468890


### Step 3: Create a new DataFrame called `hotel_df`.

In [12]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_final_narrow.copy()

hotel_df = hotel_df[["City","Country","Lat","Lng","Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
3,hirara,JP,24.8000,125.2833,94,
58,fort st. john,CA,56.2499,-120.8529,41,
72,constantia,RO,44.1833,28.6500,51,
90,colonia,DE,50.9333,6.9500,65,
111,uruzgan,AF,32.8333,66.0000,13,
148,litochoro,GR,40.1006,22.4978,46,
150,gamba,GA,-2.6500,10.0000,87,
198,dingli,MT,35.8614,14.3822,73,
203,mohammedia,MA,33.7066,-7.3944,88,
226,isola di capo rizzuto,IT,38.9600,17.0933,81,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [22]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {"categories": categories,
"apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
hirara - nearest hotel: 古謝そば屋
fort st. john - nearest hotel: Fort St. John Motor Inn
constantia - nearest hotel: Hotel Dali
colonia - nearest hotel: Wasserturm Hotel Cologne
uruzgan - nearest hotel: No hotel found
litochoro - nearest hotel: Καλλιστώ - Kallisto Traditional house
gamba - nearest hotel: No hotel found
dingli - nearest hotel: Maple farm B&B
mohammedia - nearest hotel: Novotel
isola di capo rizzuto - nearest hotel: No hotel found
morondava - nearest hotel: Hotel Sharon Menabe
saint-francois - nearest hotel: Chez Lily
vila velha - nearest hotel: Hotel Prainha
mugla - nearest hotel: İzethan Otel
devils lake - nearest hotel: Holiday Inn Express
khadan khak - nearest hotel: No hotel found
hauterive - nearest hotel: Auberge Bourbonnaise
tipasa - nearest hotel: Hôtel Munatec
yambio - nearest hotel: Kabash Inn
buka - nearest hotel: No hotel found
alora - nearest hotel: Hotel Cortijo San Antonio
nikolayevsk - nearest hotel: Опава
muynoq - nearest hotel: Lala
k

,City,Country,Lat,Lng,Humidity,Hotel Name
3,hirara,JP,24.8000,125.2833,94,古謝そば屋
58,fort st. john,CA,56.2499,-120.8529,41,Fort St. John Motor Inn
72,constantia,RO,44.1833,28.6500,51,Hotel Dali
90,colonia,DE,50.9333,6.9500,65,Wasserturm Hotel Cologne
111,uruzgan,AF,32.8333,66.0000,13,No hotel found
148,litochoro,GR,40.1006,22.4978,46,Καλλιστώ - Kallisto Traditional house
150,gamba,GA,-2.6500,10.0000,87,No hotel found
198,dingli,MT,35.8614,14.3822,73,Maple farm B&B
203,mohammedia,MA,33.7066,-7.3944,88,Novotel
226,isola di capo rizzuto,IT,38.9600,17.0933,81,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [23]:
%%capture --no-display

# Configure the map plot
map_for_hotel = hotel_df.hvplot.points("Lng","Lat", geo = True, tiles="OSM", \
                                        frame_width=700, frame_height=500, \
                                        size="Humidity", scale=0.5, color="City", \
                                        hover_cols = ["Hotel Name","Country"]
                                       )

# Display the map
map_for_hotel


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)